In [2]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
from utils import *

rootPath = "../../results/scenario/biogas"


plt.style.use('bmh')
%matplotlib inline

nBase = pypsa.Network("../../results/base/base27Node2045/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
nMean = pypsa.Network(rootPath + "-mean/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
nOptimist = pypsa.Network(rootPath + "-optimist/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")
# nBest = pypsa.Network(rootPath + "-best/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")



INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [4]:
include_biogas = True
include_Geothermie = False
include_warm = True

def getDfValue(n):
  lists =  [
    # totla 3
    n.statistics.capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_grouper").sum() / 1e9,
    n.statistics.opex(groupby="de_grouper").sum() / 1e9,
    # elec 10
    n.statistics.capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.expanded_capex(groupby="de_generator_grouper").sum() / 1e9,
    n.statistics.optimal_capacity(groupby='de_generator_grouper').sum() / 1e3,
    n.statistics.supply(groupby='de_generator_grouper').sum() / 1e6,
    n.statistics.curtailment(groupby="de_generator_grouper").sum() / 1e6,
    n.statistics.optimal_capacity(groupby="de_import_elec_grouper").sum() / 1e3,
    n.statistics.supply(groupby="de_import_elec_grouper").sum() / 1e6,
    n.statistics.optimal_capacity(groupby="de_export_elec_grouper").sum() / 1e3,
    n.statistics.supply(groupby="de_export_elec_grouper").sum() / 1e6,    
  ]

  if include_warm:
    lists.extend([
      n.statistics.capex(groupby="de_heat_generator_grouper").sum() / 1e9,
      n.statistics.expanded_capex(groupby="de_heat_generator_grouper").sum() / 1e9,
      n.statistics.opex(groupby="de_heat_generator_grouper").sum() / 1e9,
      n.statistics.optimal_capacity(groupby='de_heat_generator_grouper').sum() / 1e3,
      n.statistics.supply(groupby="de_heat_generator_grouper").sum() / 1e6,
    ])

  if include_Geothermie:
    lists.extend([
      n.statistics.supply(groupby=["carrier", "de_generator_grouper"]).groupby('carrier').sum().get('geothermal organic rankine cycle', 0),
      renameHeatCarrier(n.statistics.supply(groupby=["carrier", "de_heat_generator_grouper"]).groupby('carrier').sum()).get('Geothermie', 0),
    ])
  
  if include_biogas:
    lists.extend([
      n.statistics.supply(groupby="de_biogas_generator_grouper").sum() /1e6,
      n.statistics.supply(groupby="de_gas_generator_grouper").sum() /1e6,
      n.statistics.supply(groupby="de_biogas_generator_grouper").sum()/n.statistics.supply(groupby="de_gas_generator_grouper").sum() * 100
    ])
  
  return lists

data = {
    'Name': [
       # total
      'CAPEX', 
      'investierte CAPEX', 
      'OPEX',
       # elec
      'Erzeugung CAPEX',
      'Erzeugung investierte CAPEX',
      'Erzeugung OPEX',
      'Stromkapazität',
      'Stromerzeugung',
      'curtailment',
      'Importkapazität',
      'Importmenge',
      'Exportkapazität',
      'Exportmenge',
       # heat
       
      ],
    'Unit': [
      # total
      'Mrd.Euro',
      'Mrd.Euro',
      'Mrd.Euro',
       # elec
      'Mrd.Euro',
      'Mrd.Euro',
      'Mrd.Euro',
      'GW',
      'TWH',
      'TWH',
      'GW',
      'TWH',
      'GW',
      'TWH',
      # heat
       
    ],
    'Base': getDfValue(nBase),
    'Mean': getDfValue(nMean),
    'Optimist': getDfValue(nOptimist),
    # 'Best': getDfValue(nBest)
}

if include_warm:
  data['Name'].extend([
    'Wärm CAPEX',
    'Wärm investierte CAPEX',
    'Wärm OPEX',
    'Wärmekapazität',
    'Wärmerzeugung',
  ])
  data['Unit'].extend([
    'Mrd.Euro',
    'Mrd.Euro',
    'Mrd.Euro',
    'GW',
    'TWH',     
  ])

if include_Geothermie:
  data['Name'].extend([
    'geothermal elec menge',
    'geothermal Wärm menge'
  ])
  data['Unit'].extend([
    'MWH',
    'MWH',
  ])

if include_biogas:
  data['Name'].extend([
    'Biogasproduktion',
    'Gasproduktion',
    'biogas/Gas'
  ])
  data['Unit'].extend([
    'MWH',
    'MWH',
    '%'
  ])

print(len(data['Name']), len(data['Unit']), len(data['Base']))

df = pd.DataFrame(data).set_index('Name')

print(df)

21 21 21
                                 Unit      Base      Mean
Name                                                     
CAPEX                        Mrd.Euro  64.25519  64.02858
investierte CAPEX            Mrd.Euro  44.91728  44.69068
OPEX                         Mrd.Euro   6.63091   6.38962
Erzeugung CAPEX              Mrd.Euro  26.20495  26.20809
Erzeugung investierte CAPEX  Mrd.Euro  10.90579  10.90893
Erzeugung OPEX               Mrd.Euro  10.90579  10.90893
Stromkapazität                     GW 210.52356 210.57895
Stromerzeugung                    TWH 589.77387 591.90046
curtailment                       TWH   7.86180   5.78778
Importkapazität                    GW  92.96970  92.97435
Importmenge                       TWH 269.29716 270.13714
Exportkapazität                    GW  98.19332  98.19610
Exportmenge                       TWH 243.77392 243.40737
Wärm CAPEX                   Mrd.Euro  15.03818  15.03909
Wärm investierte CAPEX       Mrd.Euro  15.03818  15.03909
Wärm 